In [18]:
from mtcnn import MTCNN
import cv2
import matplotlib.pyplot as plt
import time
from keras_facenet import FaceNet
from numpy import asarray
import pickle

# Preparing

## Evaluation

In [19]:
import numpy as np 
from numpy.linalg import norm
def EvalCosine(vector,data,threshold):
    Name = 'Unknown'
    best = 0
    for obj in data:
        A = vector
        B = obj.vector
        # compute cosine similarity
        cosine = np.dot(A,B)/(norm(A)*norm(B))
        if(cosine>= threshold and cosine >=best):
            Name=obj.name
            best=cosine
    return Name + "_"+str(round( best, 2  ))

## Class to hold the objects

In [20]:
class ProcessedImage:
    def __init__ (self, name , vector):
        self.name = name
        self.vector=vector

## Deserializing feature vector + name

In [21]:
with open('Extracted_Features', 'rb') as f:
    Data = pickle.load(f)

## Declaring agents

In [22]:
embedder = FaceNet()
detector = MTCNN(min_face_size=250) #need to be fine-tuned accordingly to the webcam's resolution

# Deployment

In [23]:
cap = cv2.VideoCapture(0) #change to 0 to switch to the default camera input

if not cap.isOpened():
    print("Cannot open camera")
    exit()



#fps
prev_frame_time = 0
new_frame_time = 0


while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    frame=cv2.resize(frame, (420, 420))  
    if ret == True:

        detections = detector.detect_faces(frame)
        #bounding box
        img_crop = []
        for det in detections:
            cv2.rectangle(frame, (det['box'][0],det['box'][1]), (det['box'][0]+det['box'][2], det['box'][1]+det['box'][3]), (0, 255, 0), 3)
            img_crop.append(frame[det['box'][1]:det['box'][1]+det['box'][3], det['box'][0]:det['box'][0]+det['box'][2]])
        
            if (len(img_crop) != 0 ):
                #extract features
                new_trainx =list()
                
                for cropped in img_crop:
                    embedding = embedder.embeddings(img_crop)
                    new_trainx.append(embedding)

                new_trainx = asarray(new_trainx)
                Name=str(EvalCosine(new_trainx[0][0],Data,0.6))
                cv2.putText(frame, Name , (det['box'][0], det['box'][1]-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            



        #Display fps
        font = cv2.FONT_HERSHEY_SIMPLEX
        new_frame_time = time.time()
        fps = 1/(new_frame_time-prev_frame_time)
        prev_frame_time = new_frame_time
        fps = int(fps)
        fps = str(fps)
        cv2.putText(frame, fps, (7, 70), font, 2, (100, 255, 0), 2, cv2.LINE_AA)

        # Display the resulting frame
        cv2.imshow('Frame', frame)
    
        # Press Q on keyboard to  exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
   
    else:
        break
cap.release()
cv2.destroyAllWindows()